In [1]:
# 共通で利用するライブラリ
import glob
import os
import datetime
import pickle
from natsort import natsorted
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=3)
pd.options.display.float_format = "{:.4f}".format
pd.set_option("display.max_columns", None)
plt.style.use("seaborn")
plt.rcParams["font.size"] = 14
plt.rcParams["font.family"] = "IPAexGothic"
%matplotlib inline
%load_ext autoreload

# 追加するライブラリ


## Ch.09 小規模機械学習システムを作成する

- 月次で更新するタイミングで、データを蓄積する
- 蓄積されたデータをもとに毎月のモデル構築を行う
- データ一覧
  - m_area.csv
  - m_store.csv
  - tbl_order_202004.csv ~ tbl_order_202008.csv
  - store_monthly_data.csv
  - ml_base_data.csv
  - model_y_weekday_GradientBoosting.pickle
  - model_y_weekend_GradientBoosting.pickle
  - X_cols.csv

- フォルダ構成

```bash
ch09
├── ch09.ipynb
├── models
│   ├── model_y_weekday_GradientBoosting.pickle
│   ├── model_y_weekend_GradientBoosting.pickle
│   └── X_cols.csv
└── data
    ├── 00_input
    │   └── tbl_order_YYYYMM.csv
    ├── 01_store_monthly
    │   └── store_monthly_data.csv
    ├── 02_ml_base
    │   └── ml_base_data.csv
    ├── 10_output_ml_result
    │   └── results_xxxx
    ├── 11_output_report
    └── 99_master
```

### Knock81: フォルダを生成して初期の変数定義を行う

In [2]:
# フォルダ作成

data_dir = "data"

input_dir = os.path.join(data_dir, "00_input")
store_monthly_dir = os.path.join(data_dir, "01_store_monthly")
ml_base_dir = os.path.join(data_dir, "02_ml_base")

output_ml_result_dir = os.path.join(data_dir, "10_output_ml_result")
output_report_dir = os.path.join(data_dir, "11_output_report")

master_dir = os.path.join(data_dir, "99_master")
model_dir = "models"

os.makedirs(input_dir, exist_ok=True)
os.makedirs(store_monthly_dir, exist_ok=True)
os.makedirs(ml_base_dir, exist_ok=True)
os.makedirs(output_ml_result_dir, exist_ok=True)
os.makedirs(output_report_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [ ]:
# 初期変数の定義

tg_ym = "202004"

target_file = "tbl_order_" + tg_ym + ".csv"
m_area_file = "m_area.csv"
m_store_file = "m_store.csv"
store_monthly_file = "store_monthly_data.csv"
ml_base_file = "ml_base_data.csv"
